In [7]:
# FILE = "X:\\SteamLibrary\\steamapps\\common\\L.A.Noire\\final\\pc\\movies.big.pc"
IN_FILE = "..\\dump\\out\\files\\out\\trunk\\models\\la\\environment\\weather\\sun.trunk.pc"
OUT_FILE = "dump3.dds"
OFFSET = 0x320
SIZE = 0x1590F - OFFSET

with open(IN_FILE, "rb") as file:
	file.seek(OFFSET)
	with open(OUT_FILE, "wb") as out:
		out.write(file.read(SIZE))

In [4]:
from os import makedirs
from os.path import dirname, join
from struct import unpack
from json import dump

# FILE_NAME = "examples/cases.big.pc"
# FILE_NAME = "examples/audio.big.pc"
FILE_NAME = "examples/out.wad.pc"
OUT_FILE = "file_data.json"
OUT_DIRECTORY = "dump"

In [2]:
from io import IOBase

class FileStream():
	def __init__(self, file: IOBase):
		self._file = file
		self._endian = "@"

	def set_endian(self, endian: str):
		self._endian = endian

	def tell(self):
		return self._file.tell()

	def seek(self, offset: int, end: int):
		self._file.seek(offset, end)

	def read(self, format: str, length: int):
		return unpack(f"{self._endian}{format}", self._file.read(length))[0]

	def read_uint8(self):
		return self.read("B", 1)
		
	def read_int8(self):
		return self.read("b", 1)
		
	def read_uint16(self):
		return self.read("H", 2)
		
	def read_int16(self):
		return self.read("h", 2)

	def read_uint32(self):
		return self.read("I", 4)
		
	def read_int32(self):
		return self.read("i", 4)
		
	def read_uint64(self):
		return self.read("Q", 8)
		
	def read_int64(self):
		return self.read("q", 8)

	def read_float32(self):
		return self.read("f", 4)
		
	def read_float64(self):
		return self.read("d", 8)

	def read_string(self, length: int):
		return self.read(f"{length}s", length)

	def read_pad(self, length: int):
		self.read_chunk(length)

	def read_chunk(self, length: int):
		return self._file.read(length)

## BIG Files

In [7]:
with open(FILE_NAME, "rb") as big:
	stream = FileStream(big)
	stream.seek(-4, 2)

	print("Pos:", hex(stream.tell()))
	size = stream.read_uint32()
	print("Size:", size, hex(size))
	print("Pos:", hex(stream.tell()))

	stream.seek(-size, 2)
	stream.read_pad(0x4)

	print("Pos:", hex(stream.tell()))
	num_files = stream.read_uint32()
	print("Files:", num_files, hex(num_files))
	print("Pos:", hex(stream.tell()))
	print()

	file_data = [None] * num_files

	print(num_files)
	for i in range(num_files):
		hash = stream.read_uint32()
		offset = stream.read_uint32()
		size = stream.read_uint32()
		stream.read_pad(4)
		xsize = stream.read_uint32()
		offset *= 0x10
		file_data[i] = [hash, offset, size, xsize]

	for i in range(num_files):
		pass

	# stream.seek(-4, 2)
	# print("Pos:", hex(stream.tell()))
	# table_offset = unpack("<I", big.read(4))[0]
	# file_offset = table_offset - 0xC
	# size_offset = table_offset - 0x10
	# print("Size:", size_offset, hex(size_offset))
	# print("Pos:", hex(stream.tell()))

	# big.seek(-size_offset, 2)

	# print("Pos:", hex(stream.tell()))
	# size = unpack("<I", big.read(4))[0]
	# print("Files:", size, hex(num_files))
	# print("Pos:", hex(stream.tell()))

	# print(size)
	# big.seek(4, 1)
	# raw_size = unpack("<I", big.read(4))[0]
	# big.seek(-file_offset, 2)
	# offset = unpack("<I", big.read(4))[0] * 0x10

	# print(size, raw_size, offset)

	# big.seek(offset)

Pos: 0x153520f4
Size: 103352 0x193b8
Pos: 0x153520f8
Pos: 0x15338d44
Files: 5167 0x142f
Pos: 0x15338d48

5167
2593008032 0x0 294848 0
2824889955 0x48000 891264 0
3990475606 0x121980 1144448 0
2188748568 0x239000 1490816 0
163073441 0x3a4f80 1093760 0
2641786278 0x4b0000 421696 0
3623955311 0x516f40 295104 0
2677170374 0x55f000 278336 0
2462940219 0x5a2f40 614528 0
2503540759 0x639000 265824 0
1977967989 0x679e60 512384 0
1333846135 0x6f7000 332288 0
2196986484 0x748200 273920 0
2120328607 0x78b000 663264 0
3653867588 0x82cee0 311552 0
1486451095 0x879000 385568 0
3789317094 0x8d7220 2960768 0
3410583826 0xba9fa0 319584 0
1938253538 0xbf8000 53184 0
2962364469 0xc04fc0 20544 0
1251569727 0xc0a000 822848 0
448409651 0xcd2e40 1900928 0
4204886927 0xea3000 347648 0
2156083812 0xef7e00 25088 0
4093801764 0xefe000 36384 0
463716814 0xf06e20 500064 0
3339932733 0xf80f80 835712 0
2971883938 0x104d000 44544 0
2414495385 0x1057e00 713216 0
2141524109 0x1106000 2387456 0
1850139273 0x134ce00 9958

## WAD Files

In [5]:
with open(FILE_NAME, "rb") as wad:
	stream: FileStream = FileStream(wad)

	sign = stream.read_string(3)
	stream.read_pad(1)
	num_files = stream.read_uint32()
	
	file_data = [None] * num_files
	names_offset = 0

	for i in range(num_files):
		hash = stream.read_uint32()
		offset = stream.read_uint32()
		size = stream.read_uint32()
		names_offset = max(offset + size, names_offset)
		file_data[i] = [hash, offset, size]

		print(hex(offset))

	stream.seek(names_offset, 0)
	for i in range(num_files):
		name_len = stream.read_uint16()
		name = stream.read_string(name_len).decode("utf-8")
		file_data[i].append(name_len)
		file_data[i].append(name)
		offset = file_data[i][1]
		size = file_data[i][2]

		# position = stream.tell()

		# stream.seek(offset, 0)
		# makedirs(join(OUT_DIRECTORY, dirname(name)), exist_ok = True)
		# with open(join(OUT_DIRECTORY, name), "wb") as data_file:
		# 	data_file.write(stream.read_chunk(size))

		# stream.seek(position, 0)


# with open(join(OUT_DIRECTORY, FILE_NAME.split(".")[0], OUT_FILE), "w") as file:
# 	dump(file_data, file, indent = 4)

0x1a3c
0x1c00
0x423f8
0x43525
0x835a5
0x860dd
0x8675a
0xe5b306
0xe65e3e
0xe667bb
0xe712f3
0xe7be2b
0xe7e963
0xe93f38
0xe96a70
0xe995a8
0xe9c0e0
0xe9c75d
0xe9d0da
0xe9da57
0xea058f
0xea30c7
0xea5bff
0xea8737
0xeab26f
0xeabbec
0xeae724
0xeb125c
0xeb3d94
0xeb68cc
0xeb9404
0xebbf3c
0xebea74
0xec15ac
0xec40e4
0xec6c1c
0xec9754
0xecc28c
0xecedc4
0xed18fc
0xed4434
0xed6f6c
0xed9aa4
0xedc5dc
0xee7114
0xee9c4c
0xef4784
0xeff2bc
0xf01df4
0xf0492c
0xf07464
0xf09f9c
0xf0cad4
0xf0f60c
0xf12144
0xf14c7c
0xf177b4
0xf1a2ec
0xf1ce24
0xf1f95c
0xf22494
0xf24fcc
0xf27b04
0xf2a63c
0xf35174
0xf3a764
0xf3fd54
0xf45344
0xf4a934
0xf4ff24
0xf55514
0xf6004c
0xf800cc
0xf8054c
0xf8064c
0xfa06cc
0xfc074c
0xfd07cc
0xfd0c4c
0xfd10cc
0xff114c
0x10111cc
0x101324c
0x10332cc
0x103434c
0x10543cc
0x107444c
0x10748cc
0x109494c
0x109c9cc
0x10a4a4c
0x10a5cad
0x1125d2d
0x11280c4
0x1228144
0x12297c4
0x12a9844
0x12aae16
0x132ae96
0x132c865
0x13ac8e5
0x13aefc1
0x15af041
0x15b0d4c
0x1630dcc
0x163720e
0x167e76e
0x1682dc5
0x1707525


In [13]:
DIRECTORY = "X:\\SteamLibrary\\steamapps\\common\\L.A.Noire"

from os import walk
from os.path import join

for root, dirs, files in walk(DIRECTORY, ):
	if "Mods" in root:
		continue
	for file in files:
		with open("out.xml", "a") as f:
			f.write(f"\t<GameFile Name=\"{join(root, file).lstrip(DIRECTORY)}\" />\n")
